In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind
import numpy as np
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

students = pd.read_csv('stud_math.csv')

Импортируем библиотеки и читаем файл

In [ ]:
students.isna().sum()

Как можно заметить, в файле очень много пустых значений. Для Score удалим такие строки, так как это главный столбец, для количественных переменных-заменим средним

In [ ]:
sns.boxplot(x='score', y='school',
            data=students)

# Гистограммы распределения

# Score

In [ ]:
IQR = students.score.quantile(0.75) - students.score.quantile(0.25)
perc25 = students.score.quantile(0.25)
perc75 = students.score.quantile(0.75)
print(
'25-й перцентиль: {},'.format(perc25),
'75-й перцентиль: {},'.format(perc75),
"IQR: {}, ".format(IQR),
"Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))
 
students.score.loc[students.score.between(
perc25 - 1.5*IQR,
perc75 + 1.5*IQR)].hist(bins = 16, range = (0, 100), label = 'IQR')
 
students.score.hist(
alpha = 0.5, bins = 16, range = (0, 100), label = 'Здравый смысл (оценка не может быть больше 100)')

In [ ]:
students.score.loc[students.score > 100]

In [ ]:
students.score.loc[students.score < 0]

In [ ]:
students = students.dropna(subset=['score'], axis = 0)
students = students.loc[students['score'].between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)]

Границы выбросов - -5:115. Следовательно, 0 входит в удовлетворяющие значения, что правильно логически

Проверим, что пустые значения заменились

In [ ]:
students.loc[students.score.isna() == 1]

# Absences

In [ ]:
IQR = students.absences.quantile(0.75) - students.absences.quantile(0.25)
perc25 = students.absences.quantile(0.25)
perc75 = students.absences.quantile(0.75)
print(
'25-й перцентиль: {},'.format(perc25),
'75-й перцентиль: {},'.format(perc75),
"IQR: {}, ".format(IQR),
"Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))
 
students.absences.hist(bins = 16, range = (0, 102.5), label = 'IQR')

Заменим выбросы и пустые значения  таблице absences на среднее значение

In [ ]:
students['absences'] = students['absences'].apply(
    lambda x: students.absences.mean() if (x < perc25 - 1.5*IQR) or (x > perc75 + 1.5*IQR) else x)
students['absences'] = students['absences'].fillna(students.absences.mean())

Medu, Fedu, traveltime, studytime, failures, famrel, freetime, goout, health не должны иметь выбросов, так как ограничены маленьким набором возможных ввариантов. требуется проверить соответствие границам и заменить неизвестные значения средним	

# Medu

In [ ]:
students['Medu'] = students['Medu'].apply(
    lambda x: students.Medu.mean() if (x < 0) or (x > 4) else x)
students['Medu'] = students['Medu'].fillna(students.Medu.mean())

# Fedu

In [ ]:
students['Fedu'] = students['Fedu'].apply(
    lambda x: students.Fedu.mean() if (x < 0) or (x > 4) else x)
students['Fedu'] = students['Fedu'].fillna(students.Fedu.mean())

# traveltime

In [ ]:
students['traveltime'] = students['traveltime'].apply(
    lambda x: students.traveltime.mean() if (x < 0) or (x > 4) else x)
students['traveltime'] = students['traveltime'].fillna(students.traveltime.mean())

# studytime

In [ ]:
students['studytime'] = students['studytime'].apply(
    lambda x: students.studytime.mean() if (x < 0) or (x > 4) else x)
students['studytime'] = students['studytime'].fillna(students.studytime.mean())

# failures

In [ ]:
students['failures'] = students['failures'].apply(
    lambda x: students.failures.mean() if (x < 0) or (x > 4) else x)
students['failures'] = students['failures'].fillna(students.failures.mean())

# studytime, granular

Нет информации касательно границ знаений, которые могут быть представлены в этой колонке, поэтому просто заполним пустые ячейки средним

In [ ]:
students['studytime, granular'] = students['studytime, granular'].fillna(students['studytime, granular'].mean())

# famrel

In [ ]:
students['famrel'] = students['famrel'].apply(
    lambda x: students['famrel'].mean() if (x < 0) or (x > 4) else x)
students['famrel'] = students['famrel'].fillna(students.famrel.mean())

# freetime

In [ ]:
students['freetime'] = students['freetime'].apply(
    lambda x: students['freetime'].mean() if (x < 0) or (x > 4) else x)
students['freetime'] = students['freetime'].fillna(students.freetime.mean())

# goout

In [ ]:
students['goout'] = students['goout'].apply(
    lambda x: students['goout'].mean() if (x < 0) or (x > 4) else x)
students['goout'] = students['goout'].fillna(students.goout.mean())

# health

In [ ]:
students['health'] = students['health'].apply(
    lambda x: students['health'].mean() if (x < 0) or (x > 4) else x)
students['health'] = students['health'].fillna(students.health.mean())

# Корреляционный анализ

In [ ]:
students.corr()

In [ ]:
sns.heatmap(students.corr(), annot = True, cmap="YlGnBu")
fig = plt.gcf()
fig.set_size_inches(12, 12)

Studytime и studytime,granular полностью обратно скореллированны, следовательно, отбросим studytime,granular,оставив только studytime

In [ ]:
students.drop(['studytime, granular'], axis = 'columns', inplace = True)

In [ ]:
display(students)

# Количество уникальных значений

In [ ]:
display(pd.DataFrame(students.school.value_counts()))
print("Уникальных значений:", students.school.nunique())

In [ ]:
display(pd.DataFrame(students.sex.value_counts()))
print("Уникальных значений:", students.sex.nunique())

In [ ]:
display(pd.DataFrame(students.address.value_counts()))
print("Уникальных значений:", students.address.nunique())

In [ ]:
display(pd.DataFrame(students.famsize.value_counts()))
print("Уникальных значений:", students.famsize.nunique())

In [ ]:
display(pd.DataFrame(students.Pstatus.value_counts()))
print("Уникальных значений:", students.Pstatus.nunique())

In [ ]:
display(pd.DataFrame(students.Mjob.value_counts()))
print("Уникальных значений:", students.Mjob.nunique())

In [ ]:
display(pd.DataFrame(students.Fjob.value_counts()))
print("Уникальных значений:", students.Fjob.nunique())

In [ ]:
display(pd.DataFrame(students.reason.value_counts()))
print("Уникальных значений:", students.reason.nunique())

In [ ]:
display(pd.DataFrame(students.guardian.value_counts()))
print("Уникальных значений:", students.guardian.nunique())

Остальные номинативные переменные имют лишь два варианта ответа - yes and no, так что рассматривать их на этом этапе не представляется целесообразным

# Анализ номинативных переменных

Рассмотрим боксплоты распределения номинативных переменных

In [ ]:
def get_boxplot(column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=column, y='score', 
                data=students)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [ ]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']:
    get_boxplot(col)

Графики разнятся для практически всех номинативных переменных, поэтому обратимся к ttest

In [ ]:
def get_stat_dif(column):
    cols = students.loc[:, column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(students.loc[students.loc[:, column] == comb[0], 'score'],
                     students.loc[students.loc[:, column] == comb[1], 'score']).pvalue \
                <= 0.05/len(combinations_all):
            print('Найдены статистически значимые различия для колонки', column)
            break

Значения T-test без замены пустых значений на Unknown

In [ ]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']:
    get_stat_dif(col)

Заполним пустые ячейки значением unknown_value

In [ ]:
def change_unknown(column):
    students.loc[:, column] = students.loc[:, column].fillna('unknown_value')

In [ ]:
for column in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']:
    change_unknown(column)

Значения t-test с заменой

In [ ]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']:
    get_stat_dif(col)

Таким образом, для дальнейшего построения модели необходимо воспользоваться номинативными признаками: address, Mlob, higher, romantic( причем для higher  и romantic статистически важные различия были найдены также при замене пустых значений на unknown_value). При рассмотрении боксплотов видно что наибольшая разница - между вариантами ответов находящихся в колонке higher

И количественными переменными:
age, Medu, Fedu, traveltime, studytime, failures, famrel, freetime, goout, health, absences

Были отвергнуты:
'Studytime, granular', 'school', 'sex', 'famsize', 'Pstatus', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'internet'.
